# BioInf

## Read Fasta

In [271]:
def get_info(filename):
    from re import sub, search

    res = []
    inf = []
    sequence = None
    info = None
    
    fh = open(filename)

    for line in fh:
        if search(">.*", line):
                if sequence is not None and info is not None and sequence != "":
                    res.append(sequence)
                    inf.append(info)
                info = line              
                sequence = ""
        else:
            if sequence is None: return None
            else: sequence += sub("\s","",line)

    if sequence is not None and info is not None and sequence != "":
                    res.append(sequence)
                    inf.append(info)
    fh.close()

    return (inf, res)

In [272]:
def read_fasta(filename):
    dic={}
    (infos,sequencias)=get_info(filename)

    for i in range(len(infos)):
        info=infos[i]
        index=info.find("[")
        species=info[index+1:-2]
        dic[species]=sequencias[i]
    return dic


## Pre-processing of the DNA sequence

Validate DNA sequence

In [273]:
def validate_dna (dna_seq):
    seqm = dna_seq.upper()
    valid = seqm.count("A") + seqm.count("C") + seqm.count("G") + seqm.count("T")
    if valid == len(seqm): return True
    else: return False

Get the reverse complement sequence

In [274]:
complement={
    "A": "T",
    "T": "A",
    "C": "G",
    "G": "C"
}

def reverse_complement (dna_seq):
    comp=''
    for nuc in dna_seq:
        comp+=complement[nuc]
    
    return comp[::-1]

Transcription (oculted) and Translation of the DNA sequence into an aminoacids chain

In [275]:
def translate_codon (cod):
    tc = {"GCT":"A", "GCC":"A", "GCA":"A", "GCG":"A",
      "TGT":"C", "TGC":"C",
      "GAT":"D", "GAC":"D",
      "GAA":"E", "GAG":"E",
      "TTT":"F", "TTC":"F",
      "GGT":"G", "GGC":"G", "GGA":"G", "GGG":"G",
      "CAT":"H", "CAC":"H",
      "ATA":"I", "ATT":"I", "ATC":"I",
      "AAA":"K", "AAG":"K",
      "TTA":"L", "TTG":"L", "CTT":"L", "CTC":"L", "CTA":"L", "CTG":"L",
      "ATG":"M", "AAT":"N", "AAC":"N",
      "CCT":"P", "CCC":"P", "CCA":"P", "CCG":"P",
      "CAA":"Q", "CAG":"Q",
      "CGT":"R", "CGC":"R", "CGA":"R", "CGG":"R", "AGA":"R", "AGG":"R",
      "TCT":"S", "TCC":"S", "TCA":"S", "TCG":"S", "AGT":"S", "AGC":"S",
      "ACT":"T", "ACC":"T", "ACA":"T", "ACG":"T",
      "GTT":"V", "GTC":"V", "GTA":"V", "GTG":"V",
      "TGG":"W",
      "TAT":"Y", "TAC":"Y",
      "TAA":"_", "TAG":"_", "TGA":"_"}
    if cod in tc: return tc[cod]
    else: return None


def translate_seq (dna_seq, ini_pos = 0):
    seq_aa = ""
    for i in range(ini_pos, len(dna_seq)-2, 3):
        codon=dna_seq[i]+dna_seq[i+1]+dna_seq[i+2]
        seq_aa+=translate_codon(codon)
    return seq_aa

Compute the (transcription and) translation of the 6 reading frames of the DNA  sequence into aminoacids chains

In [276]:
def reading_frames (dna_seq):
    res = []
    res.append(translate_seq(dna_seq,0))
    res.append(translate_seq(dna_seq,1))
    res.append(translate_seq(dna_seq,2))
    rc = reverse_complement(dna_seq)
    res.append(translate_seq(rc,0))
    res.append(translate_seq(rc,1))
    res.append(translate_seq(rc,2))
    return res

Search for proteins in a aminoacids chain

In [277]:
def all_proteins_rf (aa_seq):
    aa_seq = aa_seq.upper()
    current_prot = []
    proteins = []
    for aa in aa_seq:
        if aa == "_":
            if current_prot:
                for p in current_prot:
                    proteins.append(p)
                current_prot = []
        else:
            if aa == "M":
                current_prot.append("")
            for i in range(len(current_prot)):
                current_prot[i] += aa
    return proteins

Search for all proteins in the six reading frames

In [278]:
def all_orfs (dna_seq):
    res = []
    translations= reading_frames(dna_seq)
    
    for trans in translations:
        res+=all_proteins_rf(trans)
                
    return res

Acording to research this protein has more than 270 aminoacids in every species to be analised

In [279]:
def all_orfs_longproteins (dna_seq, minsize = 270):
    
    all = all_orfs(dna_seq)
    res=[]
    for s in all:
        if len(s)>minsize:
            res.append(s)
    
    res2=sorted(res, key=len)

    return res2

Program that performs this analysis when given a DNA sequence and returns a list of long proteins

In [280]:
def pre_process(dna):
    assert validate_dna(dna), "Invalid DNA sequence"
    return all_orfs_longproteins(dna)

## MSA

não sei oq eq isso significa, problemas para mais tarde

## Main Project

Protein fasta - resulting from downloading by Smart-blast-p search: https://blast.ncbi.nlm.nih.gov/Blast.cgi

DNA fasta - resulting from downloading by blast search: https://blast.ncbi.nlm.nih.gov/Blast.cgi

In [281]:
def main(filename, type, n=10):
    if type=="aa":
        protein_by_species = read_fasta(filename)

    elif type=="dna":
        dna_by_species = read_fasta(filename)
        protein_by_species={}

        for species in dna_by_species.keys():
            protein=pre_process(dna_by_species[species])[0]
            protein_by_species[species]=protein

    return protein_by_species

## Testar o código

Testar o código com as 11 sequencias de aa que saquei da net atraves do link do blast

In [282]:
print(main("seqdump.txt", "aa", 12))

{'Homo sapiens': 'MLTLTRIRTVSYEVRSTFLFISVLEFAVGFLTNAFVFLVNFWDVVKRQALSNSDCVLLCLSISRLFLHGLLFLSAIQLTHFQKLSEPLNHSYQAIIMLWMIANQANLWLAACLSLLYCSKLIRFSHTFLICLASWVSRKISQMLLGIILCSCICTVLCVWCFFSRPHFTVTTVLFMNNNTRLNWQIKDLNLFYSFLFCYLWSVPPFLLFLVSSGMLTVSLGRHMRTMKVYTRNSRDPSLEAHIKALKSLVSFFCFFVISSCAAFISVPLLILWRDKIGVMVCVGIMAACPSGHAAILISGNAKLRRAVMTILLWAQSSLKVRADHKADSRTLC', 'Pan troglodytes': 'MLTLTRIHTVSYEVRSTFLFISVLEFAVGFLTNAFVFLVNFWDVVKRQPLSNSDCVLLCLSISRLFLHGLLFLSAIQLTHFQKLSEPLNHSYQAIIMLWMIANQANLWLAACLSLLYCSKLIRFSHTFLICLASWVSRKISQMLLGIILCSCICTVLCVWCFFSRPHFTVTTVLFMNNNTRLNWQIKDLNLFYSFLFCYLWSVPPFLLFLVSSGMLTVSLGRHMRTMKVYTRDSRDPSLEAHIKALKSLVSFFCFFVISSCAAFISVPLLILWRDKIGVMVCVGIMAACPSGHAAVLISGNAKLRRAVTTILLWAQSSLKVRADHKADSRTLC', 'Gorilla gorilla gorilla': 'MLTLTRIRTVSYEVRSTFLFISVLEFAVGFLTNAFVFLVNFWDVVKRQPLSNSDCVLLCLSISRLFLHGLLFLSAIQLTHFQKLSEPLNHSYQAIIMLWMIANQANLWLAACLSLLYCSKLIRFSHTFLICLASWVSRKISQMLLGIILCSCICTVLCVWCFFSRPHFTVTTVLFMNNNTRLNWQIKDLNLFYSFLFCYLWSVPPFLLFLVSSGMLTVSLGRHMRTMKVYIRDSRDPSLEAHIKALKSLVSFFCFFVISSCA

Testar usando o "fasta" do professor - mexi no documento para ficar em conta de como o blast faz o download automatico de ficheiros, mas isto dps tem de ser visto com cuidado e falado com o stor

In [283]:
print(main("sequence_Homo_sapiens.fasta", "dna", 12))

{'Homo sapiens': 'MLTLTRIRTVSYEVRSTFLFISVLEFAVGFLTNAFVFLVNFWDVVKRQALSNSDCVLLCLSISRLFLHGLLFLSAIQLTHFQKLSEPLNHSYQAIIMLWMIANQANLWLAACLSLLYCSKLIRFSHTFLICLASWVSRKISQMLLGIILCSCICTVLCVWCFFSRPHFTVTTVLFMNNNTRLNWQIKDLNLFYSFLFCYLWSVPPFLLFLVSSGMLTVSLGRHMRTMKVYTRNSRDPSLEAHIKALKSLVSFFCFFVISSCAAFISVPLLILWRDKIGVMVCVGIMAACPSGHAAILISGNAKLRRAVMTILLWAQSSLKVRADHKADSRTLC'}


## Bibliography

https://www.ncbi.nlm.nih.gov

https://blast.ncbi.nlm.nih.gov

Professor's notes and programs